In [0]:
import math
from operator import itemgetter



In [0]:
#sample dictionary ins the user based format

songData = {"Angelica": {"Blues Traveler": 3.5, "Broken Bells": 2.0, "Norah Jones": 4.5, "Phoenix": 5.0, "Slightly Stoopid": 1.5, "The Strokes": 2.5, "Vampire Weekend": 2.0},
         "Bill":{"Blues Traveler": 2.0, "Broken Bells": 3.5, "Deadmau5": 4.0, "Phoenix": 2.0, "Slightly Stoopid": 3.5, "Vampire Weekend": 3.0},
         "Chan": {"Blues Traveler": 5.0, "Broken Bells": 1.0, "Deadmau5": 1.0, "Norah Jones": 3.0, "Phoenix": 5, "Slightly Stoopid": 1.0},
         "Dan": {"Blues Traveler": 3.0, "Broken Bells": 4.0, "Deadmau5": 4.5, "Phoenix": 3.0, "Slightly Stoopid": 4.5, "The Strokes": 4.0, "Vampire Weekend": 2.0},
         "Hailey": {"Broken Bells": 4.0, "Deadmau5": 1.0, "Norah Jones": 4.0, "The Strokes": 4.0, "Vampire Weekend": 1.0},
         "Jordyn":  {"Broken Bells": 4.5, "Deadmau5": 4.0, "Norah Jones": 5.0, "Phoenix": 5.0, "Slightly Stoopid": 4.5, "The Strokes": 4.0, "Vampire Weekend": 4.0},
         "Sam": {"Blues Traveler": 5.0, "Broken Bells": 2.0, "Norah Jones": 3.0, "Phoenix": 5.0, "Slightly Stoopid": 4.0, "The Strokes": 5.0},
         "Veronica": {"Blues Traveler": 3.0, "Norah Jones": 5.0, "Phoenix": 4.0, "Slightly Stoopid": 2.5, "The Strokes": 3.0}
        }


In [9]:
#Constructing a class to make recommendations using User - Based Filtering

class similarity:
    
    # Class instantiation 
    #ratingP: Available ratings for the user we are trying to recommend a product for
    #ratingQ: Dictionary we are using to make the recommendation

    def __init__ (self, ratingP, ratingQ):
        self.ratings1 = ratingP
        self.ratings2 = ratingQ

    # Minkowski Distance between two vectors
    def minkowski(self, r):
    
  
        # calculate minkowski distance
        distance = 0       
        for k in (set(self.ratings1.keys()) & set(self.ratings2.keys())):
            p = self.ratings1[k]
            q = self.ratings2[k]
            distance += pow(abs(p - q), r)
    
        # return value of minkowski distance
        return pow(distance,1/r)
    
    # Pearson Correlation between two vectors
    def pearson(self):

          n = 0
          xiyi = 0
          xi = 0
          yi = 0
          xi2 = 0
          yi2 = 0
          for k in self.ratings1.keys():
            if k in self.ratings2.keys():
              n += 1
              a = self.ratings1[k]
              b = self.ratings2[k]

              # error check
              if n == 0:
                return (-2) 

              xiyi += a*b
              xi += a
              yi += b
              xi2 += pow(a,2)
              yi2 += pow(b,2)

          # calculating the numerator term for pearson correlation
          # using relevant partial sums

              num = (xiyi - (xi*yi)/n)


          # calculating the denominator term for pearson correlation
          # using relevant partial sums

              d1 = pow(xi2-((pow(xi,2))/n),1/2)
              d2 = pow(yi2-((pow(yi,2))/n),1/2)
              
              den = d1*d2
          # error check for denominator==0 condition
          # return -2 if denominator==0    

          if den == 0:
              
            return (-2) 

          # calcualte the pearson correlation 
          # using the numerator and deonomminator
          # and return the pearson correlation
          return num/(den)


#--------------------------------------------------------------------------------------------------
# for whom are we making recommendations?
userX = 'Veronica'
userXratings = songData[userX]

# finding the similarity measure (pearson correlation) between userX's ratings, and each of the other user's ratings.

d = {}

for i in songData.keys():
  if userX != i:
    b = similarity(userXratings, songData[i])
    c = b.pearson()
    d[i] = c

    # sorting the list of tuples by highest similarity to lowest similarity.

    d_sorted = sorted(d.items(), key = itemgetter(1), reverse =True)

# userX's NN is the user at the 0th position of the sorted list.

NN = d_sorted[0]
userXNN = NN[0]

userXrecos = {}
for j in songData[userXNN]:
  if j not in userXratings:
    a = songData[userXNN][j]
    userXrecos[j] = a
 
# sort list of tuples by highest rating to lowest rating.

userXsortedrecos = sorted(userXrecos.items(),key = itemgetter(1),reverse=True)   
    
print ("recommendation for user", userX)
print ("---------------------")
print ()
print (userXsortedrecos)



recommendation for user Veronica
---------------------

[('Broken Bells', 2.0), ('Vampire Weekend', 2.0)]


In [8]:
#Constructing a class to make recommendations using Item - Based Filtering

#Note: initiate the tranpose_dictionary function only if the dataset is not in item based format
import pandas as pd
import numpy as np

class item_based:
    
    # Class instantiation 
    # itemP: It is the item we are trying to predict the ratings for
    # userP: It is the User we are predicting the item ratings for
    #item_dictionary: Dictionary we are using to predict the ratings
    def __init__ (self,itemP, userP, item_dictionary):
        self.ratings = item_dictionary
        self.itemP = itemP
        self.userP = userP

    # Use this function only if the dataset is in User Based format
    def tranpose_dictionary(self):
        matrx = pd.DataFrame.from_dict({(i):self.ratings[i] for i in self.ratings.keys()},orient = 'Index')
        matrx1 = matrx.fillna('-2')
        songDataUnfiltered = matrx1.to_dict('dict')

        transposed_dict = {}
        for i in songDataUnfiltered.keys():
          new_dict = {}
          #new_dict = dict(filter((lambda x,y: x,y if y = -2),songDataUnfiltered[i].items()))
          for key,value in songDataUnfiltered[i].items():
            if value != "-2":
              new_dict[key] = value
              transposed_dict[i] = new_dict 

        return (transposed_dict)

    #Computing the weighted slope
    def weighted_slope(self):

        iname = self.itemP
        uname = self.userP
        
        rvj = 0
        final_n = 0
        tf = 0
        tt = 0
        n1 = 0
        for i in self.ratings.keys():
          if i != iname:
            sum_num = 0
            n = 0
            for user in (set(self.ratings[iname].keys()) & set(self.ratings[i].keys())):
              n += 1 

              if n == 0:
                return (-2)

              ruk = self.ratings[iname][user]
              ruj = self.ratings[i][user]
            
              num = ruk-ruj # numerator 
              #print ("num :", num)

              sum_num += num # deviation in ratings per user amongs itemX and itemY

              dev = sum_num/n
              #print (i, dev, n)

              for u in self.ratings[i].keys():
                if u == uname:
                  user_rating = self.ratings[i][u]
            #print ("initial n :",n)

            tt += (dev+user_rating)*n
            n1 += n
            #print ('dev :', dev, "user_ratings :", user_rating, "n :", n)
        
        tf = tt

        return (round((tf/n1),2))


#---------------------------------------------------------------------------------------------------
#return ("ratings prediction for ", itemP, "when rated by ", userX, ":", rvj)

itemX = "Norah Jones" 
userX = 'Bill'

#put songData in first 
fido = item_based(itemX,userX,songData)
item_dict = fido.tranpose_dictionary()

#use transposed dictionary
#Note: can jump to this step if the dictionary is in Item Based format
test = item_based(itemX,userX,item_dict)
fido = test.weighted_slope()
fido


3.85